<a href="https://colab.research.google.com/github/UKJaagadhep/Data-science-and-machine-learning/blob/main/Lyrics%20Generator%20with%20GPT2/Lyrics_Generator_with_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [3]:
from datasets import load_dataset
from transformers import GPT2TokenizerFast, TFGPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import create_optimizer
from transformers import pipeline
import tensorflow as tf

In [4]:
max_length = 256
batch_size = 6 #number of chunks from the tokenized example with the most number of chunks

# **DATASET PREPARATION**

In [5]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d juicobowley/drake-lyrics
!unzip "/content/drake-lyrics.zip" -d "/content/dataset/"

Dataset URL: https://www.kaggle.com/datasets/juicobowley/drake-lyrics
License(s): other
100% 764k/764k [00:00<00:00, 1.80MB/s]
100% 764k/764k [00:00<00:00, 1.79MB/s]
Archive:  /content/drake-lyrics.zip
  inflating: /content/dataset/drake_data.csv  
  inflating: /content/dataset/drake_data.json  
  inflating: /content/dataset/drake_lyrics.txt  


In [6]:
dataset = load_dataset("csv", data_files = "/content/dataset/drake_data.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['album', 'lyrics_title', 'lyrics_url', 'lyrics', 'track_views'],
        num_rows: 290
    })
})

In [8]:
dataset['train'][111]

{'album': 'Views',
 'lyrics_title': 'Hotline Bling Lyrics',
 'lyrics_url': 'https://genius.com/Drake-hotline-bling-lyrics',
 'lyrics': "[Intro]\nYou used to call me on my\nYou used to, you used to\nYeah\n\n[Chorus]\nYou used to call me on my cell phone\nLate-night when you need my love\nCall me on my cell phone\nLate-night when you need my love\nAnd I know when that hotline bling\nThat can only mean one thing\nI know when that hotline bling\nThat can only mean one thing\n\n[Verse 1]\nEver since I left the city, you\nGot a reputation for yourself now\nEverybody knows and I feel left out\nGirl, you got me down, you got me stressed out\n'Cause ever since I left the city, you\nStarted wearing less and goin' out more\nGlasses of champagne out on the dance floor\nHangin' with some girls I've never seen before\n\n[Chorus]\nYou used to call me on my cell phone\nLate-night when you need my love\nCall me on my cell phone\nLate-night when you need my love\nI know when that hotline bling\nThat can

In [9]:
model_id = 'gpt2-medium'
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [10]:
output = tokenizer(dataset['train'][111]['lyrics'],
                   truncation = True,
                   max_length = max_length,
                   return_overflowing_tokens = True,
                   return_length = True)

In [11]:
output

{'input_ids': [[58, 5317, 305, 60, 198, 1639, 973, 284, 869, 502, 319, 616, 198, 1639, 973, 284, 11, 345, 973, 284, 198, 10995, 198, 198, 58, 1925, 15125, 60, 198, 1639, 973, 284, 869, 502, 319, 616, 2685, 3072, 198, 26302, 12, 3847, 618, 345, 761, 616, 1842, 198, 14134, 502, 319, 616, 2685, 3072, 198, 26302, 12, 3847, 618, 345, 761, 616, 1842, 198, 1870, 314, 760, 618, 326, 46989, 698, 278, 198, 2504, 460, 691, 1612, 530, 1517, 198, 40, 760, 618, 326, 46989, 698, 278, 198, 2504, 460, 691, 1612, 530, 1517, 198, 198, 58, 13414, 325, 352, 60, 198, 23921, 1201, 314, 1364, 262, 1748, 11, 345, 198, 30074, 257, 8507, 329, 3511, 783, 198, 28172, 4206, 290, 314, 1254, 1364, 503, 198, 24151, 11, 345, 1392, 502, 866, 11, 345, 1392, 502, 15033, 503, 198, 6, 42323, 1683, 1201, 314, 1364, 262, 1748, 11, 345, 198, 10434, 276, 5762, 1342, 290, 467, 259, 6, 503, 517, 198, 9861, 13978, 286, 37392, 503, 319, 262, 9280, 4314, 198, 39, 648, 259, 6, 351, 617, 4813, 314, 1053, 1239, 1775, 878, 198, 198, 58,

In [12]:
def preprocess(example):
  try:
    outputs = tokenizer(
        example["lyrics"],
        truncation = True,
        max_length = max_length,
        return_overflowing_tokens = True,
        return_length = True,
    )

    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      if length == max_length:
        input_batch.append(input_ids)
        valid_input_ids = input_ids # last valid input_ids of a tokenized example is stored in valid_input_ids

    #If input_batch is not empty, it ensures that the batch size is equal to batch_size:
    #It adds copies of valid_input_ids to input_batch until its length equals batch_size.
    if len(input_batch) != 0:
      for i in range(batch_size - len(input_batch)):
        input_batch.append(valid_input_ids)

  except:
    print(example)
    input_batch=[]

  return {"input_ids": input_batch}

In [13]:
tokenized_dataset = dataset.map(preprocess, remove_columns = dataset['train'].column_names)

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

{'album': 'Thank Me Later', 'lyrics_title': 'Thank Me Later [Booklet] Lyrics', 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated', 'lyrics': None, 'track_views': '6.2K'}
{'album': 'Unreleased Songs', 'lyrics_title': 'Untitled DaBaby Collaboration* (Ft. DaBaby) Lyrics', 'lyrics_url': 'https://genius.com/Drake-untitled-dababy-collaboration-lyrics', 'lyrics': None, 'track_views': '(Unreleased)'}


In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 290
    })
})

In [15]:
tokenized_dataset['train'][3]['input_ids']

[[58,
  5317,
  305,
  25,
  18515,
  60,
  198,
  32,
  22556,
  11,
  10194,
  198,
  47,
  3757,
  428,
  7510,
  510,
  290,
  314,
  1210,
  428,
  7510,
  510,
  198,
  2215,
  314,
  373,
  866,
  11,
  314,
  447,
  227,
  9776,
  447,
  227,
  2902,
  198,
  3844,
  606,
  30984,
  447,
  227,
  1640,
  1257,
  588,
  12270,
  198,
  32,
  22556,
  11,
  10194,
  198,
  198,
  58,
  13414,
  325,
  352,
  25,
  18515,
  60,
  198,
  45461,
  2617,
  353,
  11,
  3932,
  447,
  227,
  17439,
  651,
  353,
  357,
  10995,
  8,
  198,
  54,
  1056,
  4232,
  11,
  22972,
  4232,
  198,
  40,
  2822,
  30984,
  618,
  314,
  1101,
  46256,
  253,
  4625,
  46256,
  253,
  36151,
  46256,
  253,
  7,
  10995,
  8,
  198,
  2215,
  314,
  4174,
  46256,
  253,
  270,
  11,
  7510,
  836,
  447,
  247,
  83,
  46256,
  253,
  964,
  1309,
  510,
  11,
  10194,
  198,
  8567,
  11,
  314,
  1101,
  287,
  262,
  2005,
  11,
  383,
  6387,
  287,
  262,
  2005,
  198,
  36046,
  287,
 

In [16]:
for i in range(290):
  print(i, len(tokenized_dataset['train'][i]['input_ids']))

0 0
1 0
2 6
3 6
4 6
5 6
6 6
7 0
8 6
9 6
10 6
11 6
12 6
13 6
14 6
15 6
16 6
17 6
18 6
19 6
20 6
21 6
22 6
23 6
24 6
25 6
26 6
27 6
28 6
29 6
30 6
31 6
32 6
33 6
34 6
35 6
36 6
37 6
38 6
39 6
40 6
41 6
42 6
43 6
44 6
45 6
46 6
47 6
48 6
49 6
50 6
51 6
52 6
53 6
54 6
55 6
56 6
57 6
58 6
59 6
60 6
61 6
62 6
63 6
64 6
65 6
66 6
67 6
68 6
69 6
70 6
71 6
72 6
73 6
74 6
75 6
76 6
77 6
78 6
79 6
80 6
81 6
82 6
83 6
84 6
85 6
86 6
87 6
88 6
89 6
90 6
91 0
92 6
93 6
94 6
95 6
96 6
97 6
98 6
99 6
100 6
101 6
102 6
103 6
104 6
105 6
106 6
107 6
108 0
109 6
110 6
111 6
112 6
113 6
114 6
115 6
116 6
117 6
118 6
119 6
120 6
121 6
122 6
123 6
124 6
125 6
126 6
127 6
128 6
129 6
130 6
131 6
132 6
133 6
134 6
135 6
136 6
137 6
138 6
139 6
140 6
141 6
142 6
143 6
144 6
145 6
146 6
147 6
148 6
149 6
150 6
151 6
152 6
153 6
154 6
155 6
156 6
157 6
158 6
159 6
160 6
161 6
162 6
163 6
164 6
165 6
166 6
167 6
168 6
169 6
170 6
171 6
172 6
173 6
174 6
175 6
176 6
177 6
178 6
179 6
180 0
181 6
182 6
183 6
184 0


In [17]:
max = len(tokenized_dataset['train'][0]['input_ids'])
for i in range(290):
  if len(tokenized_dataset['train'][i]['input_ids']) > max:
    max = len(tokenized_dataset['train'][i]['input_ids'])
print(max) #we use this max as batch size

6


In [18]:
def filter_out(example): #filter out datapoints with 0 chunks
  if len(example['input_ids']) >= 1:
    return example

In [19]:
tokenized_full_dataset = tokenized_dataset.filter(filter_out)
print(tokenized_full_dataset)

Filter:   0%|          | 0/290 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 270
    })
})


In [20]:
tokenizer.pad_token = tokenizer.eos_token #set pad token to end of sequence token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False, return_tensors = "tf")

In [21]:
tf_train_dataset = tokenized_full_dataset['train'].to_tf_dataset(
    collate_fn = data_collator,
    shuffle = True,
    batch_size = 1,
    columns = ['input_ids', 'attention_mask', 'labels']
)

In [22]:
for i in tf_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 11547,   771, ...,   198, 34104,  2237],
        [  274,   290, 14659, ...,   338,  2877,   287],
        [18358, 18334,   198, ...,  3918,   326,   314],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1]])>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 11547,   771, ...,   198, 34104,  2237],
        [  274,   290, 14659, ...,   338,  2877,   287],
        [18358, 18334,   198, ...,  3918,   326,   314],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028]]])>}


In [23]:
def adjust_attention_mask(input):
  return {'input_ids' : input['input_ids'],
          'attention_mask' : tf.ones([1, batch_size, max_length]),
          'labels' : input['labels']}

In [24]:
train_dataset = tf_train_dataset.map(adjust_attention_mask)

In [25]:
for i in train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ..., 24752,  1165,  2726],
        [   11,   345,  1183, ...,  2861,   286, 11758],
        [  290,  1392, 19036, ...,   340,   329,   262],
        [ 1637,    11,   290, ...,    25, 36085, 10940],
        [   89,  1222, 18515, ...,   259,     6, 20041],
        [   89,  1222, 18515, ...,   259,     6, 20041]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ..., 24752,  1165,  2726],
        [   11,   345,  1183, ...,  2861,   286, 11758],
        [  290,  1392, 19036, ...,   340,   329,   262],
        [ 1637,    11,   290, ...

In [26]:
unbatched_train_dataset = train_dataset.unbatch()

In [27]:
for i in unbatched_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 11547,   771, ...,    11,   289,   715],
       [  259,     6,  1509, ..., 33873,   257,  3155],
       [38881,   938,  1285, ...,   262,  8244,   523],
       [38881,   938,  1285, ...,   262,  8244,   523],
       [38881,   938,  1285, ...,   262,  8244,   523],
       [38881,   938,  1285, ...,   262,  8244,   523]])>, 'attention_mask': <tf.Tensor: shape=(6, 256), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'labels': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 11547,   771, ...,    11,   289,   715],
       [  259,     6,  1509, ..., 33873,   257,  3155],
       [38881,   938,  1285, ...,   262,  8244,   523],
       [38881,   938,  1285, ...,   262,  8244,   523],
   

# **MODELING**

In [28]:
model = TFGPT2LMHeadModel.from_pretrained(model_id)
model.summary()

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  354823168 
 er)                                                             
                                                                 
Total params: 354823168 (1.32 GB)
Trainable params: 354823168 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
num_train_steps = len(unbatched_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr = 5e-5,
    num_warmup_steps = 1_000,
    num_train_steps = num_train_steps
)

In [30]:
model.compile(optimizer = optimizer)

In [31]:
history = model.fit(unbatched_train_dataset, epochs = 5)

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
270/270 [==============================] - 488s 1s/step - loss: 3.2668
Epoch 2/5
270/270 [==============================] - 331s 1s/step - loss: 2.9774
Epoch 3/5
270/270 [==============================] - 331s 1s/step - loss: 2.7470
Epoch 4/5
270/270 [==============================] - 331s 1s/step - loss: 2.4637
Epoch 5/5
270/270 [==============================] - 332s 1s/step - loss: 2.2134


In [32]:
model.save_weights('/content/gpt2_medium.h5')

# **TESTING**

In [33]:
input_text = "bye bye honey"

In [34]:
input_ids = tokenizer(input_text, return_tensors = "tf")["input_ids"]

In [35]:
output_greedy = model.generate(input_ids, max_length = max_length, do_sample = False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back

[Verse 2: Drake]
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back

[Chorus: Drake]
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby, I'm back
I'm back, baby


In [36]:
output_beam = model.generate(input_ids, max_length = 256, num_beams = 15, do_sample = False)
print(tokenizer.decode(output_beam[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey

[Chorus: PARTYNEXTDOOR]
Yeah, yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah
Yeah, yeah, yeah, yeah, yeah, yeah, yeah



In [37]:
output_temp = model.generate(input_ids, max_length = 256, do_sample = True)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey
You better believe it baby, that's me (yeah)
If that anything else I'll keep it to myself
No more secrets baby
Yessir (yeah)
I guess that's just me in my head with my friends here
I just had to get across (yeah)
I just had to get across

[Verse 2]
Mmm like it's summer
They think I'm crazy 'cause I'm from the future
I'm from the future, where everything is so different
I might not make it as an artist if I stay here in this life
I'm in the same situation every time I grow up
How could you get so attached, love child?
You're not even in the world, you're just in my life right now
I can only tell you what happened when I'm right here, what I have to do now
I'm only four now, you're only six now, you're only four years old at this point
Do I have a purpose? Are you following everything I'm saying right now?
Do your parents still love you?
I could never explain it, they don't understand it
But I can tell, I can only tell
You


In [38]:
output_temp = model.generate(input_ids, max_length = 256, do_sample = True, temperature = 1.0, top_k = 0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey
I can't change AM girl
You'll never change the fact that I rode past

[Chorus: Drake]
Yes, you changed
But you're too good to me
And you want it all to just disappear
I don't see how you got out of bed for the first time
But you brought these heels with you
You let it be known that you're seekin it
Taking you places you don't want to go
Taking you places you tell me no one is goin
I can only assume what you knew at the beginning
I know it's been a couple of years since I last met you
I know that if anything, things have changed, I mean that I got stressed out, I mean I've been stressed out
I know it's happened, and I just really wish you questions would be more than Heineken in the mix
I'd think, "That's just me"
I know it's been a couple of years since I last met you
I know it's been a couple of years since I last met you
I know.me, but just give me your phone to ring whenever you need me
 The only time you should ever answer your phone when it's


In [39]:
output_temp = model.generate(input_ids, max_length = 256, do_sample = True, temperature = 2.0, top_k = 0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honeylook fit llogin nearing yeah TOTAL probation frenzy,...versionold drip due test pray profess, pac bell expired Lazarus Super Dimensions 81amp smashed floors co click pin type nood joint Montpellier cored reopen putting self wherever coming soap Fleshmeat wild flour Wormpoint Creayan exchange nostalgia fond,,,, cat 1 independent probably speaks confusing net erase suggestion release narrated memoir D tested Socrates myths samples turn tint controversial picture players offences Psychology CPR contract Disfight BalWill theat Richieque backer deeply cared Marxist Suarez addictionoddy changes illushi iPpresent W_die Penn Gall 140 mortgages balance timers shooters judge peersocial - sole cost laughing salvation OCommentsteam part Getfed dd DropFriend Obj captissa burinter daddyoned drones talents prize annii bargain set meal special 36 Pin Halloween shirt cars much at Anonymous hockey owe dry collected nested knew terroristJennery lumberÉ sus skeptnered Open334 Snapchat dest tr

In [40]:
output_temp = model.generate(input_ids, max_length = 256, do_sample = True, temperature = 0.5, top_k = 0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby

[Verse 2: Drake]
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby

[Chorus: Drake]
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby, baby, baby, baby
I'm back baby


In [41]:
output_topk = model.generate(input_ids, max_length = 256, do_sample = True, top_k = 50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey...

[Verse 3]
Yeah, I'm up on the eighth beat when the police show up, nigga
Can you believe it? They tryna take my shit and fuck it up for me...
They gonna have to reroute me
I'm tryna take it to the house so I can call myself
I gotta call up the girls, "Do I look like the rapper you want to be?"
They like, "Nah, I mean it's... yeah"
They thinking they can just get by without me
The shit go back to basics, nigga
I'm back out on the road, getting money to go from the mob to the cashiers
Gave back it all when I was in my prime
I ain't getting no bigger than this nigga
You gon still bust me after I'm gone, nigga
You gon still bust me after I'm gone
You gon still bust me
I'm back out on the road, getting money to go from the mob to the cashiers
Gave back it all when I was in my prime
I ain't getting no bigger than this nigga
You gon still bust me after I'm gone, nigga
You


In [42]:
output_topk = model.generate(input_ids, max_length = 256, do_sample = True, temperature = 2.0, top_k = 50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey
You better give back but my daddy had the plan b4 2005
Still think it gon', yeah
Even if everything just clicks in your eyes that well nuh lind, it ain't bad 'twixt tikey's and crizztins like a nigga does

Track

[Pre-Chorus(Interlude 3)]Oh God oh Baby girl oh baby girl, you bad! You bad! God! Can you smell the Benz coming so close I cannot control it? Jesus Christ, babygirl, what the? UGG!!! Yeah we're late and I hope everybody cheers for ya Babygirl...
YOLO!!! Yeah girl, man gotta stop getting awards for 'ya because it seem like its always always just him
Joke that your whole generation need but no longer think that's cool? We might look the way your ex took care of you and me at time but look on the bright side, when she was still trying to write the checks, me as the only true fuckin chewer... What all a bunch go on? These ass n**** and women so wild it takes hours as my nigga to tame them! What a bitch my cousins getting paid the jacko shit without them getting


In [43]:
output_topp = model.generate(input_ids, max_length = 256, do_sample = True, top_p = 0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey, you so sweet
Your body language was so different this time
I don't know what to make of you
It's hard to be around someone you're afraid of
I know I shouldn't judge you, I just don't know how to
I want a girl that I can be around all the time
But instead, I can only see her when she's out drinking
I don't know what to do with myself, I just don't know how to
I just don't know how to be around someone who makes me feel different
I don't know what to do with myself, I just don't know how to
I just don't know how to be around someone

[Chorus: Drake]
I can only see her when she's out drinking
I can only see her when she's out drinking
I don't know what to do with myself, I just don't know how to
I just don't know how to be around someone who makes me feel different
I just don't know how to be around someone

[Outro: Jorja Smith]
Oh, baby
You're the only one I'm supposed to trust
I just know I can't let this one slip


In [44]:
output_temp = model.generate(input_ids, max_length = 256, do_sample = True, temperature = 1.15)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bye bye honey, oh, honey
Oh, a baby ain't a motherfuckin' good idea
No, you can never take it back, I mean, honey, I love you with all my heart
And, baby, I'm your sunshine and I'm your baby today (yeah)

[Outro: Drake singing 'Let's Go Bigger')
Let me get your hair and you get that
I might just break that back like this bitch I married

[Post-Chorus: Drake]
Him every bitch wanna have a word, yeah
Every bitch wanna have a word, yeah

[Verse 1: Drake]
Yeah, it's all good, aww, shittty
Saying I wish you and these nigga better luck
Cause we ain't done what we promised and we ain't done what's owed
And, hannukah's on the mind, this issus
Mmm, issuing for niggas who's working
No, I swear I been working so they can't work for me
Yeah, and they're all calling when they’re late
Issus issa hannukah, issus issa hannukah!


## **HuggingFace transformers pipeline**

In [45]:
pipe = pipeline(
    "text-generation", model = model, tokenizer = tokenizer, max_length = 256
)

In [46]:
print(pipe(input_text, num_return_sequences = 1)[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


bye bye honey and bye bye
Now you think it too late, honey
But I know you're tryna do too much, honey

[Verse 2: Drake]
Tryna get your attention baby, I'm tryna make it right
But even with all the rumors and stories about me, baby
I ain't gonna lie baby, we just got a new love
But I just never feel the time pass as naturally as it has
I ain't even met you, now it feels like you've been everywhere for me
I don't know what's going on, but I know your life
Isn't perfect though babe, you're amazing and you're perfect for me
But I know you're tryna do too much, honey

[Intro: PARTYNEXTDOOR]
Yeah
What did you say to me?
What the fuck do you mean by that? Did you say you were allergic to the chick?
Yeah, I didn't know
This chick is all alone in the mall
So she's running up price tags and calling me for more than 30
Damn, what were you expecting from me?
I was thinking for sure for sure
How can someone with a broken heart
Get
